### Assignment 3

In [1]:
import pandas as pd
import numpy as np
greene = pd.read_csv(r"../../_data/christensen_greene_f4.csv")

In [2]:
import statsmodels.formula.api as smf
import pickle
import scipy.stats as stats

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$

In [3]:
ct = greene.COST.values
q = greene.Q.values
p1 = greene.PL.values
p2 = greene.PF.values
p3 = greene.PK.values

In [4]:
lnct = np.log(ct)
lnq = np.log(q)
lnp1 = np.log(p1)
lnp2 = np.log(p2)
lnp3 = np.log(p3)

In [5]:
lnq_2 = np.multiply(lnq,lnq)
lnqlnp1 = np.multiply(lnq,lnp1)
lnqlnp2 = np.multiply(lnq,lnp2)
lnqlnp3 = np.multiply(lnq,lnp3)

In [6]:
lnp1_2 = np.multiply(lnp1,lnp1) 
lnp2_2 = np.multiply(lnp2,lnp2) 
lnp3_2 = np.multiply(lnp3,lnp3) 

In [7]:
lnp1lnp2 = np.multiply (lnp1,lnp2)
lnp1lnp3 = np.multiply (lnp1,lnp3)
lnp2lnp3 = np.multiply (lnp2,lnp3)

In [8]:
#We have a look at the results that we should get
results = smf.ols( 'lnct ~ lnq + lnq_2 + lnqlnp1 + lnqlnp2 + lnqlnp3 + lnp1 + lnp2 + lnp3 + lnp1_2 + lnp2_2 + lnp3_2 + lnp1lnp2 + lnp1lnp3 + lnp2lnp3',
                  data = greene)

In [9]:
X_test = pd.DataFrame(results.exog, columns = results.exog_names).iloc[ : , 1: ]
y_test = pd.Series(results.endog , name = results.endog_names )
intercept_test = True

In [10]:
beta_OLS_output_test = results.fit().summary2().tables[1].iloc[ :, [0]]

In [11]:
beta_OLS_output_test

,Coef.
Intercept,-76.259261
lnq,-1.080425
lnq_2,0.026489
lnqlnp1,0.131041
lnqlnp2,0.058652
lnqlnp3,0.040144
lnp1,14.718293
lnp2,-0.894733
lnp3,6.380797
lnp1_2,-0.769264


## Question 1

.
$$
\widehat{\boldsymbol{\beta}}^{(OLS)} = \left( \mathbf{X}^\top \mathbf{X}\right)^{-1} \mathbf{X}^\top \mathbf{Y}
$$
​


In [12]:
X_input = pd.DataFrame()
y_input = pd.DataFrame()

In [13]:
y_input['lnct'] = lnct
X_input['lnq'] = lnq
X_input['lnq_2'] = lnq_2
X_input['lnqlnp1'] = lnqlnp1
X_input['lnqlnp2'] = lnqlnp2
X_input['lnqlnp3'] = lnqlnp3
X_input['lnp1'] = lnp1
X_input['lnp2'] = lnp2
X_input['lnp3'] = lnp3
X_input['lnp1_2'] = lnp1_2
X_input['lnp2_2'] = lnp2_2
X_input['lnp3_2'] = lnp3_2
X_input['lnp1lnp2'] = lnp1lnp2
X_input['lnp1lnp3'] = lnp1lnp3
X_input['lnp2lnp3'] = lnp2lnp3

In [14]:
def beta_OLS(X:pd.DataFrame, y:pd.Series, intercept:bool=True) -> pd.DataFrame:
    
    if not isinstance(X, pd.DataFrame):
        raise TypeError('X is not a DataFrame')
        
    if not isinstance(y, pd.DataFrame):
        raise TypeError('y is not a pandas Series')
        
    if not isinstance(intercept, bool):
        raise TypeError('intercept is not boolean')
    
    if intercept:
        X_ = X.copy()
        X_.insert(0, 'Intercept', 1)
        betas = np.linalg.inv(X_.T @ X_) @ X_.T @ y
        betas['Params'] = X_.columns.tolist()
        betas.set_index('Params', inplace=True)
        betas.rename(columns={'lnct':'Coefs'}, inplace=True)
        return betas
    else:
        betas = np.linalg.inv(X.T @ X) @ X.T @ y
        betas['Params'] = X.columns.tolist()
        betas.set_index('Params', inplace=True)
        betas.rename(columns={'lnct':'Coefs'}, inplace=True)
        return betas

In [15]:
beta_OLS (X_input,y_input)

,Coefs
Params,
Intercept,-76.259258
lnq,-1.080425
lnq_2,0.026489
lnqlnp1,0.131041
lnqlnp2,0.058652
lnqlnp3,0.040144
lnp1,14.718292
lnp2,-0.894733
lnp3,6.380797


In [16]:
beta_OLS (X_input,y_input, False)

,Coefs
Params,
lnq,-1.070405
lnq_2,0.025880
lnqlnp1,0.129652
lnqlnp2,0.059482
lnqlnp3,0.042202
lnp1,1.034946
lnp2,-1.331499
lnp3,-0.290481
lnp1_2,-0.150969


## Question 2

In [17]:
def var_OLS(X:pd.DataFrame, y:pd.Series, intercept:bool=True) -> pd.DataFrame:
    
    if not isinstance(X, pd.DataFrame):
        raise TypeError('X is not a DataFrame')
        
    if not isinstance(y, pd.DataFrame):
        raise TypeError('y is not a pandas Series')
        
    if not isinstance(intercept, bool):
        raise TypeError('intercept is not boolean')
    
    if intercept:
        X_ = X.copy()
        X_.insert(0, 'Intercept', 1) # Add a column of 1's at the front of the df
        names = X_.columns.tolist()
        n = X_.shape[0]
        k = X_.shape[1]
        betas = np.linalg.inv(X_.T @ X_) @ X_.T @ y
        resid = y - np.dot(X_, betas)
        resid_2 = np.square(resid)
        sigma_hat = (resid_2.sum().values) / (n - k)
        var_cov_matrix = sigma_hat * np.linalg.inv(X_.T @ X_)
        var_cov_matrix = pd.DataFrame(var_cov_matrix, columns=names)
        var_cov_matrix['Index'] = names
        var_cov_matrix.set_index('Index', inplace=True)
        return var_cov_matrix
    
    else:
        names = X.columns.tolist()
        n = X.shape[0]
        k = X.shape[1]
        betas = np.linalg.inv(X.T @ X) @ X.T @ y
        resid = y - np.dot(X, betas)
        resid_2 = np.square(resid)
        sigma_hat = (resid_2.sum().values) / (n - k)
        var_cov_matrix = sigma_hat * np.linalg.inv(X.T @ X)
        var_cov_matrix = pd.DataFrame(var_cov_matrix, columns=names)
        var_cov_matrix['Index'] = names
        var_cov_matrix.set_index('Index', inplace=True)
        return var_cov_matrix

In [18]:
var_OLS(X_input,y_input)

,Intercept,lnq,lnq_2,lnqlnp1,lnqlnp2,lnqlnp3,lnp1,lnp2,lnp3,lnp1_2,lnp2_2,lnp3_2,lnp1lnp2,lnp1lnp3,lnp2lnp3
Index,,,,,,,,,,,,,,,
Intercept,1465.361102,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932563,-8.392679,-128.192044,11.880856,-0.266944,3.540639,0.990253,10.902440,0.197435
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.015321,0.000381,-0.000898,-0.015046,0.017546,-0.003570
lnq_2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000043,0.000013,-0.000060,-0.000099,-0.000014,-0.000048
lnqlnp1,-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.001961,0.000019,0.000285,0.001850,-0.001141,-0.000085
lnqlnp2,0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.000587,-0.000170,-0.000103,-0.000170,-0.000144,-0.001375
lnqlnp3,0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000211,-0.000033,-0.000067,0.000102,-0.001537,0.002212
lnp1,-262.932563,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194721,-1.253531,19.099542,-2.388986,0.070493,-0.420154,0.038416,-1.788158,0.132157
lnp2,-8.392679,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.161643,-0.058647,-0.033147,-0.685050,0.199482,-0.367388
lnp3,-128.192044,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099542,-0.214871,20.513714,-0.705214,0.023665,-0.763050,0.036229,-1.555553,-0.072591


In [19]:
var_OLS(X_input,y_input, False)

,lnq,lnq_2,lnqlnp1,lnqlnp2,lnqlnp3,lnp1,lnp2,lnp3,lnp1_2,lnp2_2,lnp3_2,lnp1lnp2,lnp1lnp3,lnp2lnp3
Index,,,,,,,,,,,,,,
lnq,0.143916,0.000316,-0.013733,0.000162,-0.006220,-0.154803,0.152503,-0.075394,0.014043,0.000424,-0.001391,-0.015489,0.016446,-0.003671
lnq_2,0.000316,0.000006,-0.000037,-0.000005,-0.000011,-0.000620,0.001005,-0.000165,0.000053,0.000011,-0.000032,-0.000093,0.000074,-0.000048
lnqlnp1,-0.013733,-0.000037,0.001495,-0.000171,0.000356,0.017120,-0.015595,0.003160,-0.001781,0.000015,0.000357,0.001906,-0.000962,-0.000083
lnqlnp2,0.000162,-0.000005,-0.000171,0.000666,-0.000193,-0.005543,0.003468,0.009624,0.000467,-0.000171,-0.000144,-0.000184,-0.000268,-0.001406
lnqlnp3,-0.006220,-0.000011,0.000356,-0.000193,0.000905,0.006897,-0.008805,0.003804,-0.000112,-0.000026,-0.000166,0.000077,-0.001869,0.002252
lnp1,-0.154803,-0.000620,0.017120,-0.005543,0.006897,3.078406,-2.816333,-3.982649,-0.262483,0.023061,0.219585,0.220554,0.171553,0.171038
lnp2,0.152503,0.001005,-0.015595,0.003468,-0.008805,-2.816333,8.196230,-0.968641,0.234424,-0.061416,-0.013134,-0.693384,0.267324,-0.373808
lnp3,-0.075394,-0.000165,0.003160,0.009624,0.003804,-3.982649,-0.968641,9.490986,0.341030,0.000319,-0.462655,0.125391,-0.614197,-0.056460
lnp1_2,0.014043,0.000053,-0.001781,0.000467,-0.000112,-0.262483,0.234424,0.341030,0.024013,-0.000934,-0.017711,-0.021884,-0.018386,-0.008427


## Question 3

In [20]:
from scipy.stats import t

def reg_OLS(X:pd.DataFrame, y:pd.Series, intercept:bool=True) -> pd.DataFrame:
    if not isinstance(X, pd.DataFrame):
        raise TypeError('X is not a DataFrame')
        
    if not isinstance(y, pd.DataFrame):
        raise TypeError('y is not a pandas Series')
        
    if not isinstance(intercept, bool):
        raise TypeError('intercept is not boolean')
    
    if intercept:
        X_ = X.copy()
        X_.insert(0, 'Intercept', 1) # Add a column of 1's at the front of the df
        n = X_.shape[0]
        k = X_.shape[1]
        betas = beta_OLS(X, y)
        betas.reset_index(inplace=True)
        var_cov_matrix = var_OLS(X, y)
        std_err = pd.DataFrame(np.sqrt(np.diag(var_cov_matrix)), columns = ['Std.Err'])
        t_df = pd.concat([betas, std_err], axis=1)
        t_df['T_stat'] = t_df.apply(lambda x: x['Coefs']/x['Std.Err'], axis=1)
        t_df['P-Value'] = t_df['T_stat'].apply(lambda x: 1 - 2*np.abs(0.5 - np.vectorize(t.cdf)(x, n-k-1)))
        t_df['[0.025'] = t_df.apply(lambda x: x['Coefs'] - t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
        t_df['0.975]'] = t_df.apply(lambda x: x['Coefs'] + t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
        t_df.set_index('Params', inplace=True)
        return t_df
    
    else:
        n = X.shape[0]
        k = X.shape[1]
        betas = beta_OLS(X, y, False)
        betas.reset_index(inplace=True)
        var_cov_matrix = var_OLS(X, y, False)
        std_err = pd.DataFrame(np.sqrt(np.diag(var_cov_matrix)), columns = ['Std.Err'])
        t_df = pd.concat([betas, std_err], axis=1)
        t_df['T_stat'] = t_df.apply(lambda x: x['Coefs']/x['Std.Err'], axis=1)
        t_df['P-Value'] = t_df['T_stat'].apply(lambda x: 1 - 2*np.abs(0.5 - np.vectorize(t.cdf)(x, n-k-1)))
        t_df['[0.025'] = t_df.apply(lambda x: x['Coefs'] - t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
        t_df['0.975]'] = t_df.apply(lambda x: x['Coefs'] + t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
        t_df.set_index('Params', inplace=True)
        return t_df

In [21]:
reg_OLS(X_input, y_input)

,Coefs,Std.Err,T_stat,P-Value,[0.025,0.975]
Params,,,,,,
Intercept,-76.259258,38.280035,-1.992142,0.048273,-151.927105,-0.591411
lnq,-1.080425,0.375545,-2.876952,0.004635,-1.822762,-0.338088
lnq_2,0.026489,0.002357,11.240060,0.000000,0.021831,0.031148
lnqlnp1,0.131041,0.038284,3.422854,0.000810,0.055365,0.206716
lnqlnp2,0.058652,0.025549,2.295650,0.023160,0.008149,0.109155
lnqlnp3,0.040144,0.029791,1.347518,0.179960,-0.018744,0.099033
lnp1,14.718292,7.084823,2.077440,0.039562,0.713777,28.722807
lnp2,-0.894733,2.842313,-0.314790,0.753383,-6.513110,4.723645
lnp3,6.380797,4.529207,1.408811,0.161077,-2.572051,15.333644


In [22]:
reg_OLS(X_input, y_input, False)

,Coefs,Std.Err,T_stat,P-Value,[0.025,0.975]
Params,,,,,,
lnq,-1.070405,0.379362,-2.821591,0.005459,-1.820244,-0.320567
lnq_2,0.025880,0.002361,10.962820,0.000000,0.021214,0.030547
lnqlnp1,0.129652,0.038670,3.352769,0.001024,0.053218,0.206087
lnqlnp2,0.059482,0.025808,2.304809,0.022617,0.008471,0.110493
lnqlnp3,0.042202,0.030079,1.403049,0.162770,-0.017251,0.101655
lnp1,1.034946,1.754539,0.589868,0.556210,-2.433031,4.502924
lnp2,-1.331499,2.862906,-0.465086,0.642577,-6.990247,4.327250
lnp3,-0.290481,3.080744,-0.094289,0.925012,-6.379803,5.798842
lnp1_2,-0.150969,0.154961,-0.974238,0.331584,-0.457261,0.155323


# Question 4

Generate a new function named as "restricted_OLS" that uses your previos functions beta_OLS and var_OLS to return an ouput similar to reg_OLS_output. The inputs of this function should be X, y, L, r, [...]

$$
\hat{\boldsymbol{\beta}}^{(RLS)} =\hat{\boldsymbol{\beta}}^{(OLS)} - \left( \boldsymbol{X}^\top \boldsymbol{X} \right)^{-1} \boldsymbol{L}^\top \left( \boldsymbol{L}\left( \boldsymbol{X}^\top \boldsymbol{X} \right)^{-1} \boldsymbol{L}^\top \right)^{-1} \left( \boldsymbol{L}\widehat{\boldsymbol{\beta}}^{(OLS)} - \boldsymbol{r} \right)
$$

$$
\begin{aligned}
\mathbb{V}{\rm ar} \left(\widehat{\boldsymbol{\beta}}^{(RLS)} \right) 
&= \sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1} - \sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}  \mathbf{L}^\top \left( \mathbf{L}\left( \mathbf{X}^\top \mathbf{X} \right)^{-1} \mathbf{L}^\top \right)^{-1}  \mathbf{L} \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}
\end{aligned}
$$

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$

ST: 

<br>
$$
\begin{aligned} 
\beta_{1} + \beta_{2} + \beta_{3} &= 1 \\
\beta_{q1} + \beta_{q2} + \beta_{q3} &= 0 \\
\beta_{11} + \beta_{12} + \beta_{13} &= 0 \\
\beta_{21} + \beta_{22} + \beta_{23} &= 0 \\
\beta_{31} + \beta_{32} + \beta_{33} &= 0 \\
\beta_{ij} = \beta_{ji}
\end{aligned}
$$

In [23]:
L_input = np.array(( [ 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0 ], 
              [ 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1 ] ) )

r_input = np.array(  [  1,  0,  0,  0,  0  ]  )

In [24]:
def restricted_OLS(X:pd.DataFrame, y:pd.Series, L:np.ndarray, r:np.ndarray, intercept:bool=True) -> pd.DataFrame:
    
    if not isinstance(X, pd.DataFrame):
        raise TypeError('X is not a DataFrame')
        
    if not isinstance(y, pd.DataFrame):
        raise TypeError('y is not a pandas Series')
         
    if not isinstance(L, np.ndarray):
        raise TypeError('L is not a Numpy Array')
        
    if not isinstance(r, np.ndarray): 
        raise TypeError('r is not a Numpy Array')
        
    if not isinstance(intercept, bool):
        raise TypeError('intercept is not boolean')
    
    if intercept:
        X_ = X.copy()
        X_.insert(0, 'Intercept', 1)
        L0=np.zeros((5,1))
        L=np.hstack((L0,L))
        
        names = X_.columns.tolist()
        n = X_.shape[0]
        k = X_.shape[1]
        betas = beta_OLS(X, y)
        betas_values = betas.values.tolist()
        betas_values = [item for sublist in betas_values for item in sublist]
        RA_1 = np.linalg.inv(X_.T @ X_) @ np.transpose(L)
        RA_2 = np.linalg.inv(np.linalg.multi_dot([L, np.linalg.inv(X_.T @ X_), np.transpose(L)]))
        RA_3 = np.dot(L, np.array(betas_values)) - r
        RA   = RA_1.dot(RA_2).dot(RA_3)
        betas_rls = betas_values - RA
        betas_rls = pd.DataFrame(betas_rls, columns = ['Coefs'])
        
        var_cov_matrix = var_OLS(X, y) # OLS Variance Covariance Matrix
        var_cov_matrix = np.matrix(var_cov_matrix)
        resid = y - np.dot(X_, betas)
        resid_2 = np.square(resid)
        sigma_hat = (resid_2.sum().values) / (n - k)
        V1 = sigma_hat * np.linalg.inv(X_.T @ X_) @ np.transpose(L)
        V2 = L @ np.linalg.inv(X_.T @ X_)
        V = V1.dot(RA_2).dot(V2)  # OSL VC Matrix into RLS VC Matrix
        var_cov_matrix_RLS = var_cov_matrix - V
        var_cov_matrix_RLS = pd.DataFrame(var_cov_matrix_RLS, columns = names)
        var_cov_matrix_RLS['Index'] = names
        var_cov_matrix_RLS.set_index('Index', inplace=True)
        
        std_err_RLS = pd.DataFrame(np.sqrt(np.diag(var_cov_matrix_RLS)), columns = ['Std.Err'])
        t_df_RLS = pd.concat([betas_rls, std_err_RLS], axis=1)
        t_df_RLS['T_stat'] = t_df_RLS.apply(lambda x: x['Coefs']/x['Std.Err'], axis=1)
        t_df_RLS['P-Value'] = t_df_RLS['T_stat'].apply(lambda x: 1 - 2*np.abs(0.5 - np.vectorize(t.cdf)(x, n-k-1)))
        t_df_RLS['[0.025'] = t_df_RLS.apply(lambda x: x['Coefs'] - t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
        t_df_RLS['0.975]'] = t_df_RLS.apply(lambda x: x['Coefs'] + t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
        t_df_RLS['Params'] = names
        t_df_RLS.set_index('Params', inplace=True)
       
        return t_df_RLS
    
    else:
        
        names = X.columns.tolist()
        n = X.shape[0]
        k = X.shape[1]
        betas = beta_OLS(X, y, False)
        betas_values = betas.values.tolist()
        betas_values = [item for sublist in betas_values for item in sublist]
        RA_1 = np.linalg.inv(X.T @ X) @ np.transpose(L)
        RA_2 = np.linalg.inv(np.linalg.multi_dot([L, np.linalg.inv(X.T @ X), np.transpose(L)]))
        RA_3 = np.dot(L, np.array(betas_values)) - r
        RA   = RA_1.dot(RA_2).dot(RA_3)
        betas_rls = betas_values - RA
        betas_rls = pd.DataFrame(betas_rls, columns = ['Coefs'])

        
        var_cov_matrix = var_OLS(X, y, False) # OLS Variance Covariance Matrix without intercept
        var_cov_matrix = np.matrix(var_cov_matrix)
        resid = y - np.dot(X, betas)
        resid_2 = np.square(resid)
        sigma_hat = (resid_2.sum().values) / (n - k)
        V1 = sigma_hat * np.linalg.inv(X.T @ X) @ np.transpose(L)
        V2 = L @ np.linalg.inv(X.T @ X)
        V = V1.dot(RA_2).dot(V2) # # OSL VC Matrix into RLS VC Matrix
        var_cov_matrix_RLS = var_cov_matrix - V
        var_cov_matrix_RLS = pd.DataFrame(var_cov_matrix_RLS, columns = names)
        var_cov_matrix_RLS['Index'] = names
        var_cov_matrix_RLS.set_index('Index', inplace=True)
        
        std_err_RLS = pd.DataFrame(np.sqrt(np.diag(var_cov_matrix_RLS)), columns = ['Std.Err'])
        t_df_RLS = pd.concat([betas_rls, std_err_RLS], axis=1)
        t_df_RLS['T_stat'] = t_df_RLS.apply(lambda x: x['Coefs']/x['Std.Err'], axis=1)
        t_df_RLS['P-Value'] = t_df_RLS['T_stat'].apply(lambda x: 1 - 2*np.abs(0.5 - np.vectorize(t.cdf)(x, n-k-1)))
        t_df_RLS['[0.025'] = t_df_RLS.apply(lambda x: x['Coefs'] - t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
        t_df_RLS['0.975]'] = t_df_RLS.apply(lambda x: x['Coefs'] + t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
        t_df_RLS['Params'] = names
        t_df_RLS.set_index('Params', inplace=True)
        
        return t_df_RLS

In [25]:
restricted_OLS (X_input, y_input, L_input, r_input)

,Coefs,Std.Err,T_stat,P-Value,[0.025,0.975]
Params,,,,,,
Intercept,-7.111200,2.755866,-2.580387,0.010884,-12.558699,-1.663701
lnq,0.461368,0.119721,3.853679,0.000176,0.224716,0.698020
lnq_2,0.029978,0.002121,14.134763,0.000000,0.025785,0.034170
lnqlnp1,-0.004354,0.022880,-0.190311,0.849337,-0.049582,0.040873
lnqlnp2,0.036778,0.024201,1.519668,0.130818,-0.011061,0.084617
lnqlnp3,-0.032424,0.021419,-1.513780,0.132304,-0.074762,0.009915
lnp1,0.215566,1.015206,0.212338,0.832148,-1.791183,2.222316
lnp2,0.402923,0.554162,0.727085,0.468371,-0.692485,1.498331
lnp3,0.381511,0.531316,0.718048,0.473907,-0.668738,1.431760


In [26]:
restricted_OLS (X_input, y_input, L_input, r_input, False)

,Coefs,Std.Err,T_stat,P-Value,[0.025,0.975]
Params,,,,,,
lnq,0.380786,0.116762,3.261212,1.387013e-03,0.149997,0.611575
lnq_2,0.030064,0.002142,14.033257,0.000000e+00,0.025829,0.034298
lnqlnp1,0.008783,0.022536,0.389719,6.973240e-01,-0.035761,0.053326
lnqlnp2,0.018255,0.023349,0.781822,4.356120e-01,-0.027897,0.064406
lnqlnp3,-0.027038,0.021536,-1.255480,2.113523e-01,-0.069604,0.015529
lnp1,-2.340761,0.224093,-10.445489,0.000000e+00,-2.783698,-1.897824
lnp2,1.755077,0.182146,9.635545,0.000000e+00,1.395051,2.115102
lnp3,1.585684,0.256616,6.179207,6.342425e-09,1.078463,2.092905
lnp1_2,0.161786,0.028963,5.585983,1.135431e-07,0.104538,0.219033


# Question 5

In [27]:
class RegClass():
    def __init__(self, X:pd.DataFrame, y:pd.Series, intercept:bool=True):
        self.X = X
        self.y = y
        self.intercept = intercept
        
    def beta_OLS(self) -> pd.DataFrame:
    
        if not isinstance(self.X, pd.DataFrame):
            raise TypeError('X is not a DataFrame')
        
        if not isinstance(self.y, pd.DataFrame):
            raise TypeError('y is not a pandas Series')
        
        if not isinstance(self.intercept, bool):
            raise TypeError('intercept is not boolean')
    
        if self.intercept:
            X_ = self.X.copy()
            X_.insert(0, 'Intercept', 1)
            betas = np.linalg.inv(X_.T @ X_) @ X_.T @ y
            betas['Params'] = X_.columns.tolist()
            betas.set_index('Params', inplace=True)
            betas.rename(columns={'lnct':'Coefs'}, inplace=True)
            return betas
        else:
            X = self.X
            betas = np.linalg.inv(X.T @ X) @ X.T @ y
            betas['Params'] = X.columns.tolist()
            betas.set_index('Params', inplace=True)
            betas.rename(columns={'lnct':'Coefs'}, inplace=True)
            return betas
    
    def var_OLS(self) -> pd.DataFrame:
    
        if not isinstance(self.X, pd.DataFrame):
            raise TypeError('X is not a DataFrame')
        
        if not isinstance(self.y, pd.DataFrame):
            raise TypeError('y is not a pandas Series')
        
        if not isinstance(self.intercept, bool):
            raise TypeError('intercept is not boolean')
    
        if self.intercept:
            X_ = self.X.copy()
            X_.insert(0, 'Intercept', 1) # Add a column of 1's at the front of the df
            names = X_.columns.tolist()
            n = X_.shape[0]
            k = X_.shape[1]
            betas = np.linalg.inv(X_.T @ X_) @ X_.T @ y
            resid = y - np.dot(X_, betas)
            resid_2 = np.square(resid)
            sigma_hat = (resid_2.sum().values) / (n - k)
            var_cov_matrix = sigma_hat * np.linalg.inv(X_.T @ X_)
            var_cov_matrix = pd.DataFrame(var_cov_matrix, columns=names)
            var_cov_matrix['Index'] = names
            var_cov_matrix.set_index('Index', inplace=True)
            return var_cov_matrix
        else:
            X = self.X
            names = X.columns.tolist()
            n = X.shape[0]
            k = X.shape[1]
            betas = np.linalg.inv(X.T @ X) @ X.T @ y
            resid = y - np.dot(X, betas)
            resid_2 = np.square(resid)
            sigma_hat = (resid_2.sum().values) / (n - k)
            var_cov_matrix = sigma_hat * np.linalg.inv(X.T @ X)
            var_cov_matrix = pd.DataFrame(var_cov_matrix, columns=names)
            var_cov_matrix['Index'] = names
            var_cov_matrix.set_index('Index', inplace=True)
            return var_cov_matrix
    
    
    def reg_OLS(self):
        if not isinstance(self.X, pd.DataFrame):
            raise TypeError('X is not a DataFrame')
        
        if not isinstance(self.y, pd.DataFrame):
            raise TypeError('y is not a pandas Series')
        
        if not isinstance(self.intercept, bool):
            raise TypeError('intercept is not boolean')
    
        if self.intercept:
            X_ = self.X.copy()
            X_.insert(0, 'Intercept', 1) # Add a column of 1's at the front of the df
            n = X_.shape[0]
            k = X_.shape[1]
            betas = beta_OLS(self.X, self.y)
            betas.reset_index(inplace=True)
            var_cov_matrix = var_OLS(self.X, self.y)
            std_err = pd.DataFrame(np.sqrt(np.diag(var_cov_matrix)), columns = ['Std.Err'])
            t_df = pd.concat([betas, std_err], axis=1)
            t_df['T_stat'] = t_df.apply(lambda x: x['Coefs']/x['Std.Err'], axis=1)
            t_df['P-Value'] = t_df['T_stat'].apply(lambda x: 1 - 2*np.abs(0.5 - np.vectorize(t.cdf)(x, n-k-1)))
            t_df['[0.025'] = t_df.apply(lambda x: x['Coefs'] - t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
            t_df['0.975]'] = t_df.apply(lambda x: x['Coefs'] + t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
            t_df.set_index('Params', inplace=True)
            betas.set_index('Params', inplace=True)
            return t_df, betas, var_cov_matrix
    
        else:
            X = self.X
            n = X.shape[0]
            k = X.shape[1]
            betas = beta_OLS(self.X, self.y, False)
            betas.reset_index(inplace=True)
            var_cov_matrix = var_OLS(self.X, self.y, False)
            std_err = pd.DataFrame(np.sqrt(np.diag(var_cov_matrix)), columns = ['Std.Err'])
            t_df = pd.concat([betas, std_err], axis=1)
            t_df['T_stat'] = t_df.apply(lambda x: x['Coefs']/x['Std.Err'], axis=1)
            t_df['P-Value'] = t_df['T_stat'].apply(lambda x: 1 - 2*np.abs(0.5 - np.vectorize(t.cdf)(x, n-k-1)))
            t_df['[0.025'] = t_df.apply(lambda x: x['Coefs'] - t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
            t_df['0.975]'] = t_df.apply(lambda x: x['Coefs'] + t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
            t_df.set_index('Params', inplace=True)
            betas.set_index('Params', inplace=True)

            return t_df, betas, var_cov_matrix
    
    def restricted_OLS(self, L:np.ndarray, r:np.ndarray) -> pd.DataFrame:
        if not isinstance(self.X, pd.DataFrame):
            raise TypeError('X is not a DataFrame')
        
        if not isinstance(self.y, pd.DataFrame):
            raise TypeError('y is not a pandas Series')
        
        if not isinstance(self.intercept, bool):
            raise TypeError('intercept is not boolean')
    
        if not isinstance(L, np.ndarray):
            raise TypeError('L is not a np.array')
        
        if not isinstance(r, np.ndarray):
            raise TypeError('r is not a np.array')
    
        if self.intercept:
            X_ = self.X.copy()
            y = self.y
            X_.insert(0, 'Intercept', 1) # Add a column of 1's at the front of the df
            L0=np.zeros((5,1))
            L=np.hstack((L0,L))
            names = X_.columns.tolist()
            n = X_.shape[0]
            k = X_.shape[1]
            betas = beta_OLS(self.X, self.y)
            betas_values = betas.values.tolist()
            betas_values = [item for sublist in betas_values for item in sublist]
            RA_1 = np.linalg.inv(X_.T @ X_) @ np.transpose(L)
            RA_2 = np.linalg.inv(np.linalg.multi_dot([L, np.linalg.inv(X_.T @ X_), np.transpose(L)]))
            RA_3 = np.dot(L, np.array(betas_values)) - r
            RA   = RA_1.dot(RA_2).dot(RA_3)
            betas_rls = betas_values - RA
            betas_rls = pd.DataFrame(betas_rls, columns = ['Coefs'])

        
            var_cov_matrix = var_OLS(self.X, self.y)
            var_cov_matrix = np.matrix(var_cov_matrix)
            resid = y - np.dot(X_, betas)
            resid_2 = np.square(resid)
            sigma_hat = (resid_2.sum().values) / (n - k)
            V1 = sigma_hat * np.linalg.inv(X_.T @ X_) @ np.transpose(L)
            V2 = L @ np.linalg.inv(X_.T @ X_)
            V = V1.dot(RA_2).dot(V2)
            var_cov_matrix_RLS = var_cov_matrix - V
            var_cov_matrix_RLS = pd.DataFrame(var_cov_matrix_RLS, columns = names)
            var_cov_matrix_RLS['Index'] = names
            var_cov_matrix_RLS.set_index('Index', inplace=True)

            std_err_RLS = pd.DataFrame(np.sqrt(np.diag(var_cov_matrix_RLS)), columns = ['Std.Err'])
            t_df_RLS = pd.concat([betas_rls, std_err_RLS], axis=1)
            t_df_RLS['T_stat'] = t_df_RLS.apply(lambda x: x['Coefs']/x['Std.Err'], axis=1)
            t_df_RLS['P-Value'] = t_df_RLS['T_stat'].apply(lambda x: 1 - 2*np.abs(0.5 - np.vectorize(t.cdf)(x, n-k-1)))
            t_df_RLS['[0.025'] = t_df_RLS.apply(lambda x: x['Coefs'] - t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
            t_df_RLS['0.975]'] = t_df_RLS.apply(lambda x: x['Coefs'] + t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
            t_df_RLS['Params'] = names
            t_df_RLS.set_index('Params', inplace=True)
            var_cov_matrix = var_OLS(self.X, self.y)
            betas_rls['Params'] = names
            betas_rls.set_index('Params', inplace=True)

            return t_df_RLS, betas, var_cov_matrix, betas_rls, var_cov_matrix_RLS
    
        else:
            X = self.X
            y = self.y
            names = X.columns.tolist()
            n = X.shape[0]
            k = X.shape[1]
            betas = beta_OLS(self.X, self.y, False)
            betas_values = betas.values.tolist()
            betas_values = [item for sublist in betas_values for item in sublist]
            RA_1 = np.linalg.inv(X.T @ X) @ np.transpose(L)
            RA_2 = np.linalg.inv(np.linalg.multi_dot([L, np.linalg.inv(X.T @ X), np.transpose(L)]))
            RA_3 = np.dot(L, np.array(betas_values)) - r
            RA   = RA_1.dot(RA_2).dot(RA_3)
            betas_rls = betas_values - RA
            betas_rls = pd.DataFrame(betas_rls, columns = ['Coefs'])


            var_cov_matrix = var_OLS(self.X, self.y, False)
            var_cov_matrix = np.matrix(var_cov_matrix)
            resid = y - np.dot(X, betas)
            resid_2 = np.square(resid)
            sigma_hat = (resid_2.sum().values) / (n - k)
            V1 = sigma_hat * np.linalg.inv(X.T @ X) @ np.transpose(L)
            V2 = L @ np.linalg.inv(X.T @ X)
            V = V1.dot(RA_2).dot(V2)
            var_cov_matrix_RLS = var_cov_matrix - V
            var_cov_matrix_RLS = pd.DataFrame(var_cov_matrix_RLS, columns = names)
            var_cov_matrix_RLS['Index'] = names
            var_cov_matrix_RLS.set_index('Index', inplace=True)

            std_err_RLS = pd.DataFrame(np.sqrt(np.diag(var_cov_matrix_RLS)), columns = ['Std.Err'])
            t_df_RLS = pd.concat([betas_rls, std_err_RLS], axis=1)
            t_df_RLS['T_stat'] = t_df_RLS.apply(lambda x: x['Coefs']/x['Std.Err'], axis=1)
            t_df_RLS['P-Value'] = t_df_RLS['T_stat'].apply(lambda x: 1 - 2*np.abs(0.5 - np.vectorize(t.cdf)(x, n-k-1)))
            t_df_RLS['[0.025'] = t_df_RLS.apply(lambda x: x['Coefs'] - t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
            t_df_RLS['0.975]'] = t_df_RLS.apply(lambda x: x['Coefs'] + t.ppf(q = 1 - 0.05 / 2, df = n-k)*x['Std.Err'], axis=1)
            t_df_RLS['Params'] = names
            t_df_RLS.set_index('Params', inplace=True)
            var_cov_matrix = var_OLS(self.X, self.y)
            betas_rls['Params'] = names
            betas_rls.set_index('Params', inplace=True)

            return t_df_RLS, betas, var_cov_matrix, betas_rls, var_cov_matrix_RLS

### Class results: With and without intercept

In [28]:
example_intercept = RegClass(X_input, y_input)
example_no_intercept = RegClass(X_input, y_input, False)

#### With intercept: Regular OLS

In [29]:
summary, betas, var_cov_matrix = example_intercept.reg_OLS()

In [30]:
summary

,Coefs,Std.Err,T_stat,P-Value,[0.025,0.975]
Params,,,,,,
Intercept,-76.259258,38.280035,-1.992142,0.048273,-151.927105,-0.591411
lnq,-1.080425,0.375545,-2.876952,0.004635,-1.822762,-0.338088
lnq_2,0.026489,0.002357,11.240060,0.000000,0.021831,0.031148
lnqlnp1,0.131041,0.038284,3.422854,0.000810,0.055365,0.206716
lnqlnp2,0.058652,0.025549,2.295650,0.023160,0.008149,0.109155
lnqlnp3,0.040144,0.029791,1.347518,0.179960,-0.018744,0.099033
lnp1,14.718292,7.084823,2.077440,0.039562,0.713777,28.722807
lnp2,-0.894733,2.842313,-0.314790,0.753383,-6.513110,4.723645
lnp3,6.380797,4.529207,1.408811,0.161077,-2.572051,15.333644


In [31]:
betas

,Coefs
Params,
Intercept,-76.259258
lnq,-1.080425
lnq_2,0.026489
lnqlnp1,0.131041
lnqlnp2,0.058652
lnqlnp3,0.040144
lnp1,14.718292
lnp2,-0.894733
lnp3,6.380797


In [32]:
var_cov_matrix

,Intercept,lnq,lnq_2,lnqlnp1,lnqlnp2,lnqlnp3,lnp1,lnp2,lnp3,lnp1_2,lnp2_2,lnp3_2,lnp1lnp2,lnp1lnp3,lnp2lnp3
Index,,,,,,,,,,,,,,,
Intercept,1465.361102,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932563,-8.392679,-128.192044,11.880856,-0.266944,3.540639,0.990253,10.902440,0.197435
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.015321,0.000381,-0.000898,-0.015046,0.017546,-0.003570
lnq_2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000043,0.000013,-0.000060,-0.000099,-0.000014,-0.000048
lnqlnp1,-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.001961,0.000019,0.000285,0.001850,-0.001141,-0.000085
lnqlnp2,0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.000587,-0.000170,-0.000103,-0.000170,-0.000144,-0.001375
lnqlnp3,0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000211,-0.000033,-0.000067,0.000102,-0.001537,0.002212
lnp1,-262.932563,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194721,-1.253531,19.099542,-2.388986,0.070493,-0.420154,0.038416,-1.788158,0.132157
lnp2,-8.392679,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.161643,-0.058647,-0.033147,-0.685050,0.199482,-0.367388
lnp3,-128.192044,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099542,-0.214871,20.513714,-0.705214,0.023665,-0.763050,0.036229,-1.555553,-0.072591


#### With intercept: Restricted OLS

In [33]:
summary, betas, var_cov_matrix, betas_RLS, var_cov_matrix_RLS = example_intercept.restricted_OLS(L_input, r_input)

In [34]:
summary

,Coefs,Std.Err,T_stat,P-Value,[0.025,0.975]
Params,,,,,,
Intercept,-7.111200,2.755866,-2.580387,0.010884,-12.558699,-1.663701
lnq,0.461368,0.119721,3.853679,0.000176,0.224716,0.698020
lnq_2,0.029978,0.002121,14.134763,0.000000,0.025785,0.034170
lnqlnp1,-0.004354,0.022880,-0.190311,0.849337,-0.049582,0.040873
lnqlnp2,0.036778,0.024201,1.519668,0.130818,-0.011061,0.084617
lnqlnp3,-0.032424,0.021419,-1.513780,0.132304,-0.074762,0.009915
lnp1,0.215566,1.015206,0.212338,0.832148,-1.791183,2.222316
lnp2,0.402923,0.554162,0.727085,0.468371,-0.692485,1.498331
lnp3,0.381511,0.531316,0.718048,0.473907,-0.668738,1.431760


In [35]:
betas

,Coefs
Params,
Intercept,-76.259258
lnq,-1.080425
lnq_2,0.026489
lnqlnp1,0.131041
lnqlnp2,0.058652
lnqlnp3,0.040144
lnp1,14.718292
lnp2,-0.894733
lnp3,6.380797


In [36]:
var_cov_matrix

,Intercept,lnq,lnq_2,lnqlnp1,lnqlnp2,lnqlnp3,lnp1,lnp2,lnp3,lnp1_2,lnp2_2,lnp3_2,lnp1lnp2,lnp1lnp3,lnp2lnp3
Index,,,,,,,,,,,,,,,
Intercept,1465.361102,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932563,-8.392679,-128.192044,11.880856,-0.266944,3.540639,0.990253,10.902440,0.197435
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.015321,0.000381,-0.000898,-0.015046,0.017546,-0.003570
lnq_2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000043,0.000013,-0.000060,-0.000099,-0.000014,-0.000048
lnqlnp1,-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.001961,0.000019,0.000285,0.001850,-0.001141,-0.000085
lnqlnp2,0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.000587,-0.000170,-0.000103,-0.000170,-0.000144,-0.001375
lnqlnp3,0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000211,-0.000033,-0.000067,0.000102,-0.001537,0.002212
lnp1,-262.932563,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194721,-1.253531,19.099542,-2.388986,0.070493,-0.420154,0.038416,-1.788158,0.132157
lnp2,-8.392679,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.161643,-0.058647,-0.033147,-0.685050,0.199482,-0.367388
lnp3,-128.192044,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099542,-0.214871,20.513714,-0.705214,0.023665,-0.763050,0.036229,-1.555553,-0.072591


In [37]:
betas_RLS

,Coefs
Params,
Intercept,-7.111200
lnq,0.461368
lnq_2,0.029978
lnqlnp1,-0.004354
lnqlnp2,0.036778
lnqlnp3,-0.032424
lnp1,0.215566
lnp2,0.402923
lnp3,0.381511


In [38]:
var_cov_matrix_RLS

,Intercept,lnq,lnq_2,lnqlnp1,lnqlnp2,lnqlnp3,lnp1,lnp2,lnp3,lnp1_2,lnp2_2,lnp3_2,lnp1lnp2,lnp1lnp3,lnp2lnp3
Index,,,,,,,,,,,,,,,
Intercept,7.594799,-0.086062,0.000092,0.014030,-0.019783,0.005752,-2.730171,1.444107,1.286064,0.177440,-0.041141,-0.367906,-0.252103,0.074663,0.293243
lnq,-0.086062,0.014333,-0.000031,-0.002606,0.002031,0.000575,0.010912,-0.013191,0.002279,0.000713,-0.001306,-0.000589,0.000002,-0.000715,0.001304
lnq_2,0.000092,-0.000031,0.000004,-0.000007,-0.000001,0.000008,-0.000017,0.000067,-0.000050,0.000006,0.000014,-0.000022,-0.000020,0.000015,0.000007
lnqlnp1,0.014030,-0.002606,-0.000007,0.000524,-0.000325,-0.000198,-0.001233,0.001523,-0.000290,-0.000215,0.000251,0.000257,0.000110,0.000105,-0.000361
lnqlnp2,-0.019783,0.002031,-0.000001,-0.000325,0.000586,-0.000260,0.004563,-0.005478,0.000914,-0.000149,0.000026,0.000206,0.000165,-0.000016,-0.000190
lnqlnp3,0.005752,0.000575,0.000008,-0.000198,-0.000260,0.000459,-0.003331,0.003955,-0.000624,0.000364,-0.000277,-0.000463,-0.000275,-0.000089,0.000552
lnp1,-2.730171,0.010912,-0.000017,-0.001233,0.004563,-0.003331,1.030643,-0.527721,-0.502922,-0.070016,0.020404,0.138580,0.094096,-0.024080,-0.114500
lnp2,1.444107,-0.013191,0.000067,0.001523,-0.005478,0.003955,-0.527721,0.307096,0.220625,0.034923,-0.001140,-0.076622,-0.055203,0.020279,0.056342
lnp3,1.286064,0.002279,-0.000050,-0.000290,0.000914,-0.000624,-0.502922,0.220625,0.282297,0.035093,-0.019264,-0.061958,-0.038893,0.003801,0.058158


#### Without intercept: Regular OLS

In [39]:
summary, betas, var_cov_matrix = example_no_intercept.reg_OLS()

In [40]:
summary

,Coefs,Std.Err,T_stat,P-Value,[0.025,0.975]
Params,,,,,,
lnq,-1.070405,0.379362,-2.821591,0.005459,-1.820244,-0.320567
lnq_2,0.025880,0.002361,10.962820,0.000000,0.021214,0.030547
lnqlnp1,0.129652,0.038670,3.352769,0.001024,0.053218,0.206087
lnqlnp2,0.059482,0.025808,2.304809,0.022617,0.008471,0.110493
lnqlnp3,0.042202,0.030079,1.403049,0.162770,-0.017251,0.101655
lnp1,1.034946,1.754539,0.589868,0.556210,-2.433031,4.502924
lnp2,-1.331499,2.862906,-0.465086,0.642577,-6.990247,4.327250
lnp3,-0.290481,3.080744,-0.094289,0.925012,-6.379803,5.798842
lnp1_2,-0.150969,0.154961,-0.974238,0.331584,-0.457261,0.155323


In [41]:
betas

,Coefs
Params,
lnq,-1.070405
lnq_2,0.025880
lnqlnp1,0.129652
lnqlnp2,0.059482
lnqlnp3,0.042202
lnp1,1.034946
lnp2,-1.331499
lnp3,-0.290481
lnp1_2,-0.150969


In [42]:
var_cov_matrix

,lnq,lnq_2,lnqlnp1,lnqlnp2,lnqlnp3,lnp1,lnp2,lnp3,lnp1_2,lnp2_2,lnp3_2,lnp1lnp2,lnp1lnp3,lnp2lnp3
Index,,,,,,,,,,,,,,
lnq,0.143916,0.000316,-0.013733,0.000162,-0.006220,-0.154803,0.152503,-0.075394,0.014043,0.000424,-0.001391,-0.015489,0.016446,-0.003671
lnq_2,0.000316,0.000006,-0.000037,-0.000005,-0.000011,-0.000620,0.001005,-0.000165,0.000053,0.000011,-0.000032,-0.000093,0.000074,-0.000048
lnqlnp1,-0.013733,-0.000037,0.001495,-0.000171,0.000356,0.017120,-0.015595,0.003160,-0.001781,0.000015,0.000357,0.001906,-0.000962,-0.000083
lnqlnp2,0.000162,-0.000005,-0.000171,0.000666,-0.000193,-0.005543,0.003468,0.009624,0.000467,-0.000171,-0.000144,-0.000184,-0.000268,-0.001406
lnqlnp3,-0.006220,-0.000011,0.000356,-0.000193,0.000905,0.006897,-0.008805,0.003804,-0.000112,-0.000026,-0.000166,0.000077,-0.001869,0.002252
lnp1,-0.154803,-0.000620,0.017120,-0.005543,0.006897,3.078406,-2.816333,-3.982649,-0.262483,0.023061,0.219585,0.220554,0.171553,0.171038
lnp2,0.152503,0.001005,-0.015595,0.003468,-0.008805,-2.816333,8.196230,-0.968641,0.234424,-0.061416,-0.013134,-0.693384,0.267324,-0.373808
lnp3,-0.075394,-0.000165,0.003160,0.009624,0.003804,-3.982649,-0.968641,9.490986,0.341030,0.000319,-0.462655,0.125391,-0.614197,-0.056460
lnp1_2,0.014043,0.000053,-0.001781,0.000467,-0.000112,-0.262483,0.234424,0.341030,0.024013,-0.000934,-0.017711,-0.021884,-0.018386,-0.008427


#### Without intercept: Restricted OLS

In [43]:
summary, betas, var_cov_matrix, betas_RLS, var_cov_matrix_RLS = example_no_intercept.restricted_OLS(L_input, r_input)  

In [44]:
summary

,Coefs,Std.Err,T_stat,P-Value,[0.025,0.975]
Params,,,,,,
lnq,0.380786,0.116762,3.261212,1.387013e-03,0.149997,0.611575
lnq_2,0.030064,0.002142,14.033257,0.000000e+00,0.025829,0.034298
lnqlnp1,0.008783,0.022536,0.389719,6.973240e-01,-0.035761,0.053326
lnqlnp2,0.018255,0.023349,0.781822,4.356120e-01,-0.027897,0.064406
lnqlnp3,-0.027038,0.021536,-1.255480,2.113523e-01,-0.069604,0.015529
lnp1,-2.340761,0.224093,-10.445489,0.000000e+00,-2.783698,-1.897824
lnp2,1.755077,0.182146,9.635545,0.000000e+00,1.395051,2.115102
lnp3,1.585684,0.256616,6.179207,6.342425e-09,1.078463,2.092905
lnp1_2,0.161786,0.028963,5.585983,1.135431e-07,0.104538,0.219033


In [45]:
betas

,Coefs
Params,
lnq,-1.070405
lnq_2,0.025880
lnqlnp1,0.129652
lnqlnp2,0.059482
lnqlnp3,0.042202
lnp1,1.034946
lnp2,-1.331499
lnp3,-0.290481
lnp1_2,-0.150969


In [46]:
var_cov_matrix

,Intercept,lnq,lnq_2,lnqlnp1,lnqlnp2,lnqlnp3,lnp1,lnp2,lnp3,lnp1_2,lnp2_2,lnp3_2,lnp1lnp2,lnp1lnp3,lnp2lnp3
Index,,,,,,,,,,,,,,,
Intercept,1465.361102,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932563,-8.392679,-128.192044,11.880856,-0.266944,3.540639,0.990253,10.902440,0.197435
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.015321,0.000381,-0.000898,-0.015046,0.017546,-0.003570
lnq_2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000043,0.000013,-0.000060,-0.000099,-0.000014,-0.000048
lnqlnp1,-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.001961,0.000019,0.000285,0.001850,-0.001141,-0.000085
lnqlnp2,0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.000587,-0.000170,-0.000103,-0.000170,-0.000144,-0.001375
lnqlnp3,0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000211,-0.000033,-0.000067,0.000102,-0.001537,0.002212
lnp1,-262.932563,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194721,-1.253531,19.099542,-2.388986,0.070493,-0.420154,0.038416,-1.788158,0.132157
lnp2,-8.392679,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.161643,-0.058647,-0.033147,-0.685050,0.199482,-0.367388
lnp3,-128.192044,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099542,-0.214871,20.513714,-0.705214,0.023665,-0.763050,0.036229,-1.555553,-0.072591


In [47]:
betas_RLS

,Coefs
Params,
lnq,0.380786
lnq_2,0.030064
lnqlnp1,0.008783
lnqlnp2,0.018255
lnqlnp3,-0.027038
lnp1,-2.340761
lnp2,1.755077
lnp3,1.585684
lnp1_2,0.161786


In [48]:
var_cov_matrix_RLS

,lnq,lnq_2,lnqlnp1,lnqlnp2,lnqlnp3,lnp1,lnp2,lnp3,lnp1_2,lnp2_2,lnp3_2,lnp1lnp2,lnp1lnp3,lnp2lnp3
Index,,,,,,,,,,,,,,
lnq,0.013633,-3.075913e-05,-0.002498,1.844404e-03,0.000653,-0.020438,0.003239,0.017200,0.002780,-0.001809,-0.004856,-0.002914,0.000134,0.004723
lnq_2,-0.000031,4.589589e-06,-0.000008,-8.500359e-07,0.000008,0.000017,0.000051,-0.000067,0.000004,0.000015,-0.000017,-0.000018,0.000014,0.000003
lnqlnp1,-0.002498,-7.641392e-06,0.000508,-2.946300e-04,-0.000213,0.003890,-0.001169,-0.002721,-0.000554,0.000334,0.000956,0.000588,-0.000034,-0.000922
lnqlnp2,0.001844,-8.500359e-07,-0.000295,5.451870e-04,-0.000251,-0.002601,-0.001752,0.004352,0.000320,-0.000083,-0.000768,-0.000502,0.000182,0.000585
lnqlnp3,0.000653,8.491427e-06,-0.000213,-2.505571e-04,0.000464,-0.001289,0.002920,-0.001631,0.000234,-0.000251,-0.000188,-0.000086,-0.000148,0.000337
lnp1,-0.020438,1.656888e-05,0.003890,-2.600664e-03,-0.001289,0.050218,-0.008772,-0.041446,-0.006358,0.005731,0.006456,0.003542,0.002816,-0.009272
lnp2,0.003239,5.067479e-05,-0.001169,-1.751625e-03,0.002920,-0.008772,0.033177,-0.024406,0.001208,0.006821,-0.006804,-0.007416,0.006208,0.000596
lnp3,0.017200,-6.724367e-05,-0.002721,4.352289e-03,-0.001631,-0.041446,-0.024406,0.065852,0.005150,-0.012551,0.000348,0.003875,-0.009024,0.008676
lnp1_2,0.002780,3.500881e-06,-0.000554,3.196896e-04,0.000234,-0.006358,0.001208,0.005150,0.000839,-0.000904,-0.000807,-0.000371,-0.000468,0.001275
